# TODO:

- prepare xgboost model quickly for first sub
- Set up kaggle kernel for submission (need whl for category_encoders and lifelines)
- Set up Competition scoring DONE
- Improve on first model:
    - Set up CV, 
    - Tune hyper parameter 
    - try different preprocessing steps  
    - try stratification on race
- try other targets with xgboost
- try cox estimator
- try NN
- ensemble everything

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import TargetEncoder, OneHotEncoder
from sklearn.preprocessing import QuantileTransformer, PowerTransformer, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import linear_model
from sklearn import metrics
import joblib

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import validation_curve, ValidationCurveDisplay
from category_encoders import CountEncoder
from category_encoders.hashing import HashingEncoder
from lifelines import KaplanMeierFitter
from tqdm.auto import tqdm

import os, glob
import gc
import time
import ast

: 

In [2]:
COMP_DATA_BASE = os.path.join("..", "data", "comp")
PREP_DATA_BASE = os.path.join("..", "data", "preprocessed")

TRAIN_PATH = os.path.join(COMP_DATA_BASE, "train.csv")
TEST_PATH = os.path.join(COMP_DATA_BASE, "test.csv")
DATA_DICT_PATH = os.path.join(COMP_DATA_BASE, "data_dictionary.csv")
SAMPLE_PATH = os.path.join(COMP_DATA_BASE, "sample_submission.csv")

features = list(pl.read_csv(TEST_PATH).select(pl.all().exclude("ID")).columns)
train_ds = pl.read_csv(TRAIN_PATH)

: 

In [3]:
num_features_dict = pl.read_csv(DATA_DICT_PATH).filter(pl.col("variable").is_in(features) & (pl.col("type") == "Numerical"))
num_features = num_features_dict.select("variable").to_series().to_list()
cat_features_dict = pl.read_csv(DATA_DICT_PATH).filter(pl.col("variable").is_in(features) & (pl.col("type") == "Categorical"))
cat_features = cat_features_dict.select("variable").to_series().to_list()
cat_features_categories = {key: train_ds.select(key).to_series().unique().to_list()  for key in cat_features}
target_columns = pl.read_csv(DATA_DICT_PATH).filter(pl.col("variable").is_in(features).not_())
display(num_features_dict)
display(cat_features_dict)
print(cat_features_categories)
display(target_columns)


variable,description,type,values
str,str,str,str
"""hla_match_c_high""","""Recipient / 1st donor allele l…","""Numerical""",null
"""hla_high_res_8""","""Recipient / 1st donor allele-l…","""Numerical""",null
"""hla_low_res_6""","""Recipient / 1st donor antigen-…","""Numerical""",null
"""hla_high_res_6""","""Recipient / 1st donor allele-l…","""Numerical""",null
"""hla_high_res_10""","""Recipient / 1st donor allele-l…","""Numerical""",null
…,…,…,…
"""comorbidity_score""","""Sorror comorbidity score""","""Numerical""",null
"""karnofsky_score""","""KPS at HCT""","""Numerical""",null
"""hla_low_res_8""","""Recipient / 1st donor antigen-…","""Numerical""",null


variable,description,type,values
str,str,str,str
"""dri_score""","""Refined disease risk index""","""Categorical""","""['Intermediate' 'High' 'N/A - …"
"""psych_disturb""","""Psychiatric disturbance""","""Categorical""","""['Yes' 'No' nan 'Not done']"""
"""cyto_score""","""Cytogenetic score""","""Categorical""","""['Intermediate' 'Favorable' 'P…"
"""diabetes""","""Diabetes""","""Categorical""","""['No' 'Yes' nan 'Not done']"""
"""tbi_status""","""TBI""","""Categorical""","""['No TBI' 'TBI + Cy +- Other' …"
…,…,…,…
"""tce_div_match""","""T-cell epitope matching""","""Categorical""","""['Permissive mismatched' 'Bi-d…"
"""donor_related""","""Related vs. unrelated donor""","""Categorical""","""['Unrelated' 'Related' 'Multip…"
"""melphalan_dose""","""Melphalan dose (mg/m^2)""","""Categorical""","""['N/A, Mel not given' 'MEL' na…"


{'dri_score': ['Missing disease status', None, 'High - TED AML case <missing cytogenetics', 'Intermediate - TED AML case <missing cytogenetics', 'Low', 'N/A - pediatric', 'N/A - non-malignant indication', 'Very high', 'High', 'TBD cytogenetics', 'N/A - disease not classifiable', 'Intermediate'], 'psych_disturb': ['Yes', 'No', 'Not done', None], 'cyto_score': ['Not tested', 'Other', 'Intermediate', 'Normal', 'Poor', 'TBD', None, 'Favorable'], 'diabetes': ['Not done', 'No', None, 'Yes'], 'tbi_status': ['TBI + Cy +- Other', 'TBI +- Other, -cGy, unknown dose', 'No TBI', 'TBI +- Other, >cGy', 'TBI +- Other, -cGy, fractionated', 'TBI +- Other, -cGy, single', 'TBI +- Other, <=cGy', 'TBI +- Other, unknown dose'], 'arrhythmia': ['No', 'Yes', None, 'Not done'], 'graft_type': ['Peripheral blood', 'Bone marrow'], 'vent_hist': ['Yes', None, 'No'], 'renal_issue': ['Not done', 'No', 'Yes', None], 'pulm_severe': [None, 'No', 'Not done', 'Yes'], 'prim_disease_hct': ['Other leukemia', 'SAA', 'PCD', 'IEA

variable,description,type,values
str,str,str,str
"""efs""","""Event-free survival""","""Categorical""","""['Event' 'Censoring']"""
"""efs_time""","""Time to event-free survival, m…","""Numerical""",null


In [4]:
"""
To evaluate the equitable prediction of transplant survival outcomes,
we use the concordance index (C-index) between a series of event
times and a predicted score across each race group.
 
It represents the global assessment of the model discrimination power:
this is the model’s ability to correctly provide a reliable ranking
of the survival times based on the individual risk scores.
 
The concordance index is a value between 0 and 1 where:
 
0.5 is the expected result from random predictions,
1.0 is perfect concordance (with no censoring, otherwise <1.0),
0.0 is perfect anti-concordance (with no censoring, otherwise >0.0)

"""

import pandas as pd
import pandas.api.types
import numpy as np
from lifelines.utils import concordance_index

class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    >>> import pandas as pd
    >>> row_id_column_name = "id"
    >>> y_pred = {'prediction': {0: 1.0, 1: 0.0, 2: 1.0}}
    >>> y_pred = pd.DataFrame(y_pred)
    >>> y_pred.insert(0, row_id_column_name, range(len(y_pred)))
    >>> y_true = { 'efs': {0: 1.0, 1: 0.0, 2: 0.0}, 'efs_time': {0: 25.1234,1: 250.1234,2: 2500.1234}, 'race_group': {0: 'race_group_1', 1: 'race_group_1', 2: 'race_group_1'}}
    >>> y_true = pd.DataFrame(y_true)
    >>> y_true.insert(0, row_id_column_name, range(len(y_true)))
    >>> score(y_true.copy(), y_pred.copy(), row_id_column_name)
    0.75
    """
    
    del solution[row_id_column_name]
    del submission[row_id_column_name]
    
    event_label = 'efs'
    interval_label = 'efs_time'
    prediction_label = 'prediction'
    for col in submission.columns:
        if not pandas.api.types.is_numeric_dtype(submission[col]):
            raise ParticipantVisibleError(f'Submission column {col} must be a number')
    # Merging solution and submission dfs on ID
    merged_df = pd.concat([solution, submission], axis=1)
    merged_df.reset_index(inplace=True)
    merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)
    metric_list = []
    for race in merged_df_race_dict.keys():
        # Retrieving values from y_test based on index
        indices = sorted(merged_df_race_dict[race])
        merged_df_race = merged_df.iloc[indices]
        # Calculate the concordance index
        c_index_race = concordance_index(
                        merged_df_race[interval_label],
                        -merged_df_race[prediction_label],
                        merged_df_race[event_label])
        metric_list.append(c_index_race)
    return float(np.mean(metric_list)-np.sqrt(np.var(metric_list)))

train = train_ds.to_pandas()
y_true = train[["ID","efs","efs_time","race_group"]].copy()
y_pred = train[["ID"]].copy()
y_pred["prediction"] = train["efs"]
m = score(y_true.copy(), y_pred.copy(), "ID")
print(f"CV Score = {m}")

CV Score = 0.7909730905427049


In [9]:
# Transform target for xgboost
# survival AFT -> Cannot use sklearn API
def transform_target_aft(data):
    return data.with_columns(pl.col("efs_time").alias("label_lower_bound"), 
                             pl.when(pl.col("efs").cast(pl.Int64) == 1)
                                .then(pl.col("efs_time"))
                                .otherwise(pl.lit(+np.inf))
                                .alias("label_upper_bound")
    )

# transform_target(train_ds)

# Survival Cox
def transform_target_cox(data):
    data_pd = data.to_pandas()
    data["efs_time2"] = data.efs_time.copy()
    data.loc[data.efs==0,"efs_time2"] *= -1
    return data.with_columns(pl.Series(data["efs_time2"]).alias("efs_time2"))

# Transform target with Kaplan Meier Fitter
# eval_metric="mae",
# objective='reg:logistic',
# (pd.Series(oof_xgb).rank(pct=True))*0.5+pd.Series(oof_cat).rank(pct=True)*0.5
def transform_target_km_filter(data):
    def transform_survival_probability(df, time_col='efs_time', event_col='efs'):
        kmf = KaplanMeierFitter()
        kmf.fit(df[time_col], df[event_col])
        y = kmf.survival_function_at_times(df[time_col]).values
        
        # Adjust for censoring
        # censored_mask = df[event_col] == 0
        # y[censored_mask] = y[censored_mask] * 1.2  # Increase survival prob for censored
        return y

    t = transform_survival_probability(data.to_pandas(), time_col='efs_time', event_col='efs')
    return data.with_columns(pl.Series(t).alias("target_km"))


# Transform target #1
# XGBRegressor
# eval_metric="mae",
# objective='reg:logistic'
# -pred_xgb
def transform_target_1(data):
    train = data.to_pandas()
    train["y"] = train.efs_time.values
    mx = train.loc[train.efs==1,"efs_time"].max()
    mn = train.loc[train.efs==0,"efs_time"].min()
    train.loc[train.efs==0,"y"] = train.loc[train.efs==0,"y"] + mx - mn
    train.y = train.y.rank()
    train.loc[train.efs==0,"y"] += len(train)//2
    train.y = train.y / train.y.max()
    return data.with_columns(pl.Series(train.y).alias("target1"))



# transform target #2
# pred
def transform_target_2(data):
    train = data.to_pandas()
    train["y"] = train.efs_time.values
    mx = train.loc[train.efs==1,"efs_time"].max()
    mn = train.loc[train.efs==0,"efs_time"].min()
    train.loc[train.efs==0,"y"] = train.loc[train.efs==0,"y"] + mx - mn
    train.y = train.y.rank()
    train.loc[train.efs==0,"y"] += 2*len(train)
    train.y = train.y / train.y.max()
    train.y = np.log( train.y )
    train.y -= train.y.mean()
    train.y *= -1.0
    return data.with_columns(pl.Series(train.y).alias("target2"))

display(train_ds.pipe(transform_target_xgb).pipe(transform_target_km_filter).pipe(transform_target_1).pipe(transform_target_2))


ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,vent_hist,renal_issue,pulm_severe,prim_disease_hct,hla_high_res_6,cmv_status,hla_high_res_10,hla_match_dqb1_high,tce_imm_match,hla_nmdp_6,hla_match_c_low,rituximab,hla_match_drb1_low,hla_match_dqb1_low,prod_type,cyto_score_detail,conditioning_intensity,ethnicity,year_hct,obesity,mrd_hct,in_vivo_tcd,tce_match,hla_match_a_high,hepatic_severe,donor_age,prior_tumor,hla_match_b_low,peptic_ulcer,age_at_hct,hla_match_a_low,gvhd_proph,rheum_issue,sex_match,hla_match_b_high,race_group,comorbidity_score,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time,label_lower_bound,label_upper_bound,target_km,target1,target2
i64,str,str,str,str,f64,f64,str,str,f64,str,str,str,str,str,f64,str,f64,f64,str,f64,f64,str,f64,f64,str,str,str,str,i64,str,str,str,str,f64,str,f64,str,f64,str,f64,f64,str,str,str,f64,str,f64,f64,str,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64
0,"""N/A - non-malignant indication""","""No""",null,"""No""",null,null,"""No TBI""","""No""",6.0,"""Bone marrow""","""No""","""No""","""No""","""IEA""",6.0,"""+/+""",null,2.0,null,6.0,2.0,"""No""",2.0,2.0,"""BM""",null,null,"""Not Hispanic or Latino""",2016,"""No""",null,"""Yes""",null,2.0,"""No""",null,"""No""",2.0,"""No""",9.942,2.0,"""FKalone""","""No""","""M-F""",2.0,"""More than one race""",0.0,90.0,"""No""",null,"""Unrelated""","""N/A, Mel not given""",8.0,"""No""",2.0,"""No""",10.0,0.0,42.356,42.356,inf,0.458687,0.87537,-1.437594
1,"""Intermediate""","""No""","""Intermediate""","""No""",2.0,8.0,"""TBI +- Other, >cGy""","""No""",6.0,"""Peripheral blood""","""No""","""No""","""No""","""AML""",6.0,"""+/+""",10.0,2.0,"""P/P""",6.0,2.0,"""No""",2.0,2.0,"""PB""","""Intermediate""","""MAC""","""Not Hispanic or Latino""",2008,"""No""","""Positive""","""No""","""Permissive""",2.0,"""No""",72.29,"""No""",2.0,"""No""",43.705,2.0,"""Other GVHD Prophylaxis""","""No""","""F-F""",2.0,"""Asian""",3.0,90.0,"""No""","""Permissive mismatched""","""Related""","""N/A, Mel not given""",8.0,"""No""",2.0,"""Yes""",10.0,1.0,4.672,4.672,4.672,0.847759,0.101458,1.47932
2,"""N/A - non-malignant indication""","""No""",null,"""No""",2.0,8.0,"""No TBI""","""No""",6.0,"""Bone marrow""","""No""","""No""","""No""","""HIS""",6.0,"""+/+""",10.0,2.0,"""P/P""",6.0,2.0,"""No""",2.0,2.0,"""BM""",null,null,"""Not Hispanic or Latino""",2019,"""No""",null,"""Yes""",null,2.0,"""No""",null,"""No""",2.0,"""No""",33.997,2.0,"""Cyclophosphamide alone""","""No""","""F-M""",2.0,"""More than one race""",0.0,90.0,"""No""","""Permissive mismatched""","""Related""","""N/A, Mel not given""",8.0,"""No""",2.0,"""No""",10.0,0.0,19.793,19.793,inf,0.462424,0.726134,-1.354672
3,"""High""","""No""","""Intermediate""","""No""",2.0,8.0,"""No TBI""","""No""",6.0,"""Bone marrow""","""No""","""No""","""No""","""ALL""",6.0,"""+/+""",10.0,2.0,"""P/P""",6.0,2.0,"""No""",2.0,2.0,"""BM""","""Intermediate""","""MAC""","""Not Hispanic or Latino""",2009,"""No""","""Positive""","""No""","""Permissive""",2.0,"""No""",29.23,"""No""",2.0,"""No""",43.245,2.0,"""FK+ MMF +- others""","""No""","""M-M""",2.0,"""White""",0.0,90.0,"""Yes""","""Permissive mismatched""","""Unrelated""","""N/A, Mel not given""",8.0,"""No""",2.0,"""No""",10.0,0.0,102.349,102.349,inf,0.456661,0.990463,-1.497155
4,"""High""","""No""",null,"""No""",2.0,8.0,"""No TBI""","""No""",6.0,"""Peripheral blood""","""No""","""No""","""No""","""MPN""",6.0,"""+/+""",10.0,2.0,null,5.0,2.0,"""No""",2.0,2.0,"""PB""",null,"""MAC""","""Hispanic or Latino""",2018,"""No""",null,"""Yes""",null,2.0,"""No""",56.81,"""No""",2.0,"""No""",29.74,2.0,"""TDEPLETION +- other""","""No""","""M-F""",2.0,"""American Indian or Alaska Nati…",1.0,90.0,"""No""","""Permissive mismatched""","""Related""","""MEL""",8.0,"""No""",2.0,"""No""",10.0,0.0,16.223,16.223,inf,0.464674,0.711134

In [113]:
# Numerical (try nothing, winsorize, quantile)
# 1. Nothing (Just winsorize )
# 2. num_transformer = QuantileTransformer(subsample=None, random_state=24).set_output(transform="pandas")
# 3. num_transformer = StandardScaler().set_output("pandas")
# 4.# num_transformer = PowerTransformer().set_output("pandas")
# 5. # num_transformer = MinMaxScaler().set_output("pandas")

# high cardinality (target encoding best)
# 1. Nothing
# 2. target_encoder = TargetEncoder().set_output(transform="pandas")
# 3. Frequency encoding
# 4. Collapse rare categories into one + ohe -> ohe
# 5. Hash encoding

# low cardinality (ohe)
# 1. Nothing 
# 2. target_encoder = TargetEncoder().set_output(transform="pandas")
# 3. ohe_encoder = OneHotEncoder(min_frequency=0.01, handle_unknown="infrequent_if_exist").set_output(transform="pandas")
# 4. Frequency encoding
NUM_CAT_CUTOFF = 10
high_cardinality_features = [col for col in train_ds.columns if col in cat_features_categories and len(cat_features_categories[col]) > NUM_CAT_CUTOFF ]
low_cardinality_features = [col for col in train_ds.columns if col in cat_features_categories and len(cat_features_categories[col]) <= NUM_CAT_CUTOFF]
numeric_features = num_features
passthrough_features = ["ID"]

low_card_feat_cats = [cat_features_categories[col] for col in low_cardinality_features]
high_card_feat_cats = [cat_features_categories[col] for col in high_cardinality_features]

target_cols = ["efs", "efs_time"]

def create_preprocessor (numeric_type=None, low_cardinality_type=None, high_cardinality_type=None, impute=False, seed=24):
    assert numeric_type in [None, "quantile", "robust", "minmax", "standard"], "Numeric type undefined"
    assert low_cardinality_type in [None, "target", "ohe", "frequency"], "low cardinality cat type undefined"
    assert high_cardinality_type in [None, "target", "ohe", "frequency"], "high cardinality cat type undefined"

    # steps for numeric features with imputation
    imputer = make_pipeline('passthrough')
    if impute:
        imputer = SimpleImputer(missing_values=np.nan, strategy='mean', add_indicator=True, keep_empty_features=True)
    
    num_transfomers = {"quantile": QuantileTransformer(subsample=None, random_state=seed), 
                        "robust": RobustScaler(), 
                        "minmax": MinMaxScaler(), 
                        "standard": StandardScaler()}
    num_encoder = "passthrough"
    if numeric_type is not None:
        num_encoder = num_transfomers[numeric_type]
        
    numeric_pipeline = make_pipeline(imputer, num_encoder)
    
    # Steps for low cardinality features
    low_card_transformers = {"target": TargetEncoder(categories=low_card_feat_cats, random_state=seed), 
                                "ohe": OneHotEncoder(handle_unknown="infrequent_if_exist", sparse_output=False, categories=low_card_feat_cats), 
                                "frequency": CountEncoder()
        }
    
    low_card_encoder = "passthrough"
    if low_cardinality_type is not None: 
        low_card_encoder = low_card_transformers[low_cardinality_type]
    
    # Steps for high cardinality features
    high_card_transformers = {"target": TargetEncoder(categories=high_card_feat_cats, random_state=seed), 
                                "ohe": OneHotEncoder(max_categories=NUM_CAT_CUTOFF, handle_unknown="infrequent_if_exist", sparse_output=False, categories=high_card_feat_cats), 
                                "frequency": CountEncoder(),
        }
    high_card_encoder = "passthrough"
    if high_cardinality_type is not None: 
        high_card_encoder = high_card_transformers[high_cardinality_type]
    
    
    
    mixed_encoded_preprocessor = ColumnTransformer(
    [
        ("special", "passthrough", passthrough_features),
        ("numerical", numeric_pipeline, numeric_features),
        (
            "high_cardinality",
            high_card_encoder,
            high_cardinality_features,
        ),
        (
            "low_cardinality",
            low_card_encoder,
            low_cardinality_features,
        ),
    ],
    verbose_feature_names_out=False,
    )
    
    mixed_encoded_preprocessor.set_output(transform="pandas")
    
    return mixed_encoded_preprocessor

mixed_encoded_preprocessor = create_preprocessor()

In [114]:
all_preprocessor_types = [{"numeric_type": type} for type in [None, "quantile", "robust", "minmax", "standard"] ] \
                        + [{"low_cardinality_type": type} for type in [None, "target", "ohe", "frequency"] ]  \
                        + [{"high_cardinality_type": type} for type in [None, "target", "frequency", "ohe"] ] 
for proc_type in tqdm(all_preprocessor_types):
    print(proc_type)
    mixed_encoded_preprocessor = create_preprocessor(**proc_type, impute=True)
    out = mixed_encoded_preprocessor.fit_transform(X, y)
    print(out.shape)
    

  0%|          | 0/13 [00:00<?, ?it/s]

{'numeric_type': None}
(28800, 78)
{'numeric_type': 'quantile'}
(28800, 78)
{'numeric_type': 'robust'}
(28800, 78)
{'numeric_type': 'minmax'}
(28800, 78)
{'numeric_type': 'standard'}
(28800, 78)
{'low_cardinality_type': None}
(28800, 78)
{'low_cardinality_type': 'target'}
(28800, 78)
{'low_cardinality_type': 'ohe'}
(28800, 189)
{'low_cardinality_type': 'frequency'}
(28800, 78)
{'high_cardinality_type': None}
(28800, 78)
{'high_cardinality_type': 'target'}
(28800, 78)
{'high_cardinality_type': 'frequency'}
(28800, 78)
{'high_cardinality_type': 'ohe'}
(28800, 105)


In [ ]:

# https://www.kaggle.com/code/riteshsinha/useful-features-in-predicting-stock-prices#Building-Features
def get_model_data(df, SECURITY_CODE):
    df_data = df.copy()
    df_stock = df_data[df_data['SecuritiesCode'] == SECURITY_CODE]
    # feature 'average price, vwap
    # Some optimization here.
    test_length = -1
    if len(df_stock['typ'].unique() == 2): # This means this is test data creation
        test_length = len(df_stock[df_stock['typ'] == 'test'])
    
    df_stock = df_stock.sort_values(by = "Date", ascending = False).reset_index(drop = True)
    if test_length > 0:
            df_stock = df_stock[0: test_length + 125]
    df_stock['average'] = (df_stock['High'] + df_stock['Low'] + df_stock['Close'])/3
    df_stock['vwap'] = (df_stock['Close'] * df_stock['Volume'])/ df_stock['Volume']
    df_stock['vwap_pct_ret'] = df_stock['vwap'].pct_change()
    df_stock['pvwap'] = df_stock['vwap_pct_ret'].shift(-1)
    df_stock['20D-EMA'] = df_stock['Close'].ewm(span=20,adjust=False).mean()
    df_stock['50D-EMA'] = df_stock['Close'].ewm(span=50,adjust=False).mean()
    df_stock['100D-EMA'] = df_stock['Close'].ewm(span=100,adjust=False).mean()
    df_stock = df_stock.sort_values(by = "Date").reset_index(drop = True)
    df_stock['rsi'] = RSI( df_stock['Close'], 5 )
    df_stock['rsicat'] = list(map(rsi_class, df_stock['rsi']))
    df_stock['dt'] = pd.to_datetime(df_stock['Date'], format  = "%Y-%m-%d")
    df_stock['dayofweek'] = df_stock['dt'].dt.dayofweek
    # Another feature day of the week will also be added.
    df_stock['rsicat'] = pd.Series(df_stock['rsicat'], dtype="category")
    df_stock = df_stock.sort_values(by = "Date").reset_index(drop = True)
    df_model = df_stock[['Open', 'High', 'Low', 'Close'
                        , 'average', 'vwap', 'rsi', 'pvwap'
                        , '20D-EMA' , '50D-EMA', '100D-EMA'
                        , 'rsicat', 'dayofweek', 'Target'
                        , 'typ', 'RowId', 'Date', 'SecuritiesCode', 'dtint']]
    # '
    return(df_model)

# Feature Engineering

In [ ]:
# Groupby(featuesX)[featureY].mean()   or std, min, max
# Ratios between columns (/, +, *, ^)
# Combining categorical features
# diff() with mean and/or std